In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re

import pandas as pd

import kraft

In [ ]:
gene_x_cell_line = pd.read_csv(
    "protein_quant_current_normalized.csv.gz", index_col=1
).drop(columns=["Protein_Id", "Description", "Group_ID", "Uniprot", "Uniprot_Acc",])

gene_x_cell_line

In [ ]:
gene_x_cell_line = gene_x_cell_line.loc[~gene_x_cell_line.index.isna()]

gene_x_cell_line = gene_x_cell_line.groupby(level=0).median()

In [ ]:
gene_x_cell_line = gene_x_cell_line.loc[
    :, ~gene_x_cell_line.columns.str.startswith("TenPx")
]

gene_x_cell_line.columns = kraft.normalize_cell_line_names(
    (re.sub("_TenPx[0-9][0-9]", "", str_) for str_ in gene_x_cell_line.columns)
)

In [ ]:
kraft.error_index(gene_x_cell_line, index_name="Gene", column_name="Cell Line")

gene_x_cell_line.to_csv("protein.gene_x_cell_line.tsv", sep="\t")

gene_x_cell_line